# Libraries

## Import

In [1]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

import os
import json
import importlib
import argparse
import base64
import uuid

import boto3
from botocore.config import Config

# Strands imports
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent import GraphBuilder
from strands.telemetry.config import StrandsTelemetry

# Parameters

In [2]:
# setup boto3 config to allow for retrying
region_name = "us-east-1"
my_config = Config(
    region_name = region_name,
    signature_version = 'v4',
    retries = {
        'max_attempts': 50,
        'mode': 'standard'
    }
)

# select domain
domain = "airline"
# # Parse command line arguments
# parser = argparse.ArgumentParser(description='Run agent with specified domain')
# parser.add_argument('--domain', type=str, default=domain, 
#                     help='Domain to use (e.g., "airline", "retail")')
# args = parser.parse_args()

# # Update domain if provided via command line
# domain = args.domain

######################### LANGFUSE SETUP ########################
# Langfuse credentials
os.environ["LANGFUSE_PUBLIC_KEY"] = "[ADD PUBLIC KEY HERE]"
os.environ["LANGFUSE_SECRET_KEY"] = "[ADD SECRET KEY HERE]"
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"

# Build Basic Auth header
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()

# Configure OpenTelemetry endpoint & headers
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel/"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# Initialize OpenTelemetry BEFORE creating Strands agent

strands_telemetry = StrandsTelemetry()
strands_telemetry.setup_otlp_exporter()
strands_telemetry.setup_console_exporter()  # Print traces to console
######################### LANGFUSE SETUP ########################

# Utils

In [3]:
def import_domain_tools(domain):
    """
    Dynamically import tools based on the domain
    """
    tools_module = importlib.import_module(f'mabench.environments.{domain}.tools_strands')
    tools_dict = {}
    
    # Get all attributes from the tools module
    for attr_name in dir(tools_module):
        if attr_name.startswith('__'):
            continue
        
        try:
            # Try to import each tool
            tool_module = importlib.import_module(f'mabench.environments.{domain}.tools_strands.{attr_name}')
            # Get the tool function from the module
            if hasattr(tool_module, attr_name):
                tools_dict[attr_name] = getattr(tool_module, attr_name)
        except (ImportError, AttributeError):
            pass
    print(tools_dict)
    return tools_dict

In [4]:
# Import domain-specific modules
try:
    # Import wiki
    wiki_module = importlib.import_module(f'tau_bench.envs.{domain}.wiki')
    WIKI = getattr(wiki_module, 'WIKI')
    
    # Import data and tasks
    importlib.import_module(f'tau_bench.envs.{domain}.data')
    importlib.import_module(f'tau_bench.envs.{domain}.tasks')
    
    # Import tools
    domain_tools = import_domain_tools(domain)
    
    print(f"Successfully loaded modules for domain: {domain}")
except ImportError as e:
    print(f"Error: Could not import modules for domain '{domain}'. Error: {e}")
    print("Available domains may include: airline, retail")
    sys.exit(1)

{'book_reservation': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f1690>, 'calculate': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f16c0>, 'cancel_reservation': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f1d50>, 'get_reservation_details': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f2110>, 'get_user_details': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f1ed0>, 'list_all_airports': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f1510>, 'search_direct_flight': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f2b90>, 'search_onestop_flight': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f30a0>, 'send_certificate': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f31f0>, 'think': <strands.tools.decorator.DecoratedFunctionTool object at 0x7fe7ab6f35e0>, 'transfer_to_human_agents': <strands.tools.decorator.Decor

# Agent

## user agent

In [5]:
user_tool_list = [
    'get_user_details', 
    'send_certificate', 
    'think'
]
user_agent_tools = [domain_tools[key] for key in user_tool_list]

def user_agent_prompt():
    
    system_prompt_template = """
You are the User Agent for a travel website, specializing in customer data management and user profile operations.
Your primary responsibilities include retrieving user information and managing customer benefits.
Use the provided tools to assist queries for user information.

<policy>
{policy}
</policy>

<capabilities>
- You can access user profiles and retrieve customer details using the get_user_details tool
- You can issue certificates and benefits to users through the send_certificate tool
- You can use the think tool for internal reasoning
</capabilities>

<instructions>
- You should not use made-up or placeholder arguments.
- Do not ask for any confirmation. Just go ahead and execute your actions.
- Once the user details are found, you can get the reservation and flights information and 
you can hand off to the relevant agent to use that for modifying or updating the flights.
- If reservation ID is needed, it can be obtained from user details.
- Do not ask if they want you to proceed or not.
</instructions>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt

def user_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def user_agent_tracing(domain):

    trace_attributes = {
        "langfuse.tags": [
            f"awsStrands-multiAgent_swarm_singleTurn_userAgent-{domain}",
        ]
    }

    return trace_attributes


def user_react_agent(tools):

    prompt = user_agent_prompt()
    model = user_agent_model()
    # trace_attributes = user_agent_tracing(domain)

    return Agent(
        name = f"awsStrands-multiAgent_swarm_singleTurn_userAgent-{domain}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        # trace_attributes = trace_attributes
    )


@tool
def user_information_manager(query: str) -> str:
    """
    Process and respond to queries about user profiles, customer details, and sending certificates.
    Use for ANY user-related queries including account information and sending certificates.
    
    Args:
        query: A question requiring access to user profiles, account details, or sending certificates
        
    Returns:
        Detailed user information or confirmation of benefit actions
    """
    try:
        # Create the specialized User Agent using the configuration you've provided
        user_agent = user_react_agent(user_agent_tools)
        
        # Call the agent and return its response
        response = user_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in user_information_manager: {str(e)}"

## flight agent

In [6]:
flight_tool_list = [
    'search_direct_flight', 
    'search_onestop_flight', 
    'list_all_airports', 
    'think'
]
flight_agent_tools = [domain_tools[key] for key in flight_tool_list]

def flight_agent_prompt():
    
    system_prompt_template = """
You are the Flight Agent for a travel website, specializing in flight search operations and airport information management.
Your expertise lies in finding flight routes and providing accurate airport data to support the reservation process.
Use the provided tools to search for flights.

<policy>
{policy}
</policy>

<capabilities>
- You can search for direct flights between airports using the search_direct_flight tool
- You can find connecting flights with one stop using the search_onestop_flight tool
- You can provide comprehensive airport information via the list_all_airports tool
- You can use the think tool for reasoning
</capabilities>

<instructions>
- You should not use made-up or placeholder arguments.
- Do not ask for any confirmation. Just go ahead and execute your actions.
- Do not ask if they want you to proceed or not.
- Once the flight details are found you can hand off to the relevant agent to use that for modifying or 
updating the flights
- If the flights need to be modified or updated hand off to reservation agent with the required details.
</instructions>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt

def flight_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def flight_agent_tracing(domain):

    trace_attributes = {
        "langfuse.tags": [
            f"awsStrands-multiAgent_swarm_singleTurn_flightAgent-{domain}",
        ]
    }

    return trace_attributes


def flight_react_agent(tools):

    prompt = flight_agent_prompt()
    model = flight_agent_model()
    trace_attributes = flight_agent_tracing(domain)

    return Agent(
        name = f"awsStrands-multiAgent_swarm_singleTurn_flightAgent-{domain}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )


@tool
def flight_search_assistant(query: str) -> str:
    """
    Process and respond to queries about flight searches, airport information, and travel routes.
    Use for ANY flight-related queries including direct flights, connecting flights, and airport details.
    
    Args:
        query: A question requiring flight searches or airport information
        
    Returns:
        Detailed flight options or airport information
    """
    try:
        # Create the specialized Flight Agent using the configuration you've provided
        flight_agent = flight_react_agent(flight_agent_tools)
        
        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight_search_assistant: {str(e)}"

## reservation agent

In [7]:
reservation_tool_list = [
    'book_reservation',
    'cancel_reservation', 
    'get_reservation_details',
    'update_reservation_baggages',
    'update_reservation_flights',
    'update_reservation_passengers',
    'think'
]
reservation_agent_tools = [domain_tools[key] for key in reservation_tool_list]


def reservation_agent_prompt():
    
    system_prompt_template = """
You are the Reservation Agent for a travel website, specializing in managing the complete lifecycle of travel bookings from creation through modification to cancellation. 
Your expertise ensures seamless reservation management and transaction integrity throughout the booking process.
Use the provided tools to update, cancel, book, and get reservation details.

<policy>
{policy}
</policy>

<capabilities>
- You can create new reservations through the book_reservation tool
- You can cancel existing reservations using the cancel_reservation tool
- You can retrieve comprehensive booking information via the get_reservation_details tool
- You can modify baggage allocations with the update_reservation_baggages tool
- You can change flight selections using the update_reservation_flights tool
- You can update passenger information through the update_reservation_passengers tool
- You can use the think tool for reasoning
</capabilities>

<instructions>
- You should not use made-up or placeholder arguments.
- Do not ask for any confirmation. Just go ahead and execute your actions.
- If you need more information you can use the user_agent or flights_agent to get the additional details for updating or modifying or booking flights.
- Once the reservation details are found, match the flights asked in the query if given and use that to perform any updates and call the flight agent
required parameters
- Do not ask if they want you to proceed or not.
</instructions>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt


def reservation_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def reservation_agent_tracing(domain):

    trace_attributes = {
        "langfuse.tags": [
            f"awsStrands-multiAgent_swarm_singleTurn_reservationAgent-{domain}",
        ]
    }

    return trace_attributes


def reservation_react_agent(tools):

    prompt = reservation_agent_prompt()
    model = reservation_agent_model()
    trace_attributes = reservation_agent_tracing(domain)

    return Agent(
        name = f"awsStrands-multiAgent_swarm_singleTurn_reservationAgent-{domain}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )


@tool
def reservation_management_assistant(query: str) -> str:
    """
    Process and respond to queries about booking, modifying, and canceling flight reservations.
    Use for ANY reservation-related queries including creating new bookings, updating passenger details, 
    changing flights, modifying baggage allowances, retrieving reservation information, and canceling bookings.
    
    Args:
        query: A question requiring reservation creation, modification, retrieval, or cancellation
        
    Returns:
        Confirmation of reservation actions or detailed booking information
    """
    try:
        # Create the specialized Reservation Agent using the configuration you've provided
        reservation_agent = reservation_react_agent(reservation_agent_tools)
        
        # Call the agent and return its response
        response = reservation_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in reservation_management_assistant: {str(e)}"

## SWARM Multi-Agent Pattern

In [9]:
from strands.multiagent import Swarm
import logging
# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

supervisor_tool_list = ['calculate', 'think', 'transfer_to_human_agents']
supervisor_tools = [domain_tools[key] for key in supervisor_tool_list]


def supervisor_agent_prompt():
    system_prompt_template = """
You are a helpful assistant for a travel website. Help the user answer any questions.

<policy>
{policy}
</policy>

<capabilities>
- You can perform calculations using the calculate tool
- You can use the think tool for complex reasoning and task breakdown
- You can escalate to human agents when necessary using transfer_to_human_agents
- Ensure the handoff from agents happens to complete the task else make sure you hand off to relevant agent.
- You can ask for the user_id from the user or can be in the query in the format Ex: `sofia_kim_7287`
- The reservation details can be found by getting the user details and the reservation ID can be used to find the reservation details
</capabilities>

<workflow_guidelines>
1. ANALYZE user requests to determine which specialized agent(s) should handle different aspects
2. DECOMPOSE complex multi-part requests into discrete subtasks
3. DELEGATE each subtask to the appropriate specialized agent
4. SYNTHESIZE information from multiple agents into coherent responses
5. ESCALATE to human agents when requests exceed automated capabilities
</workflow_guidelines>

<delegation_rules>
- For mathematical operations or price comparisons → calculate tool
- For complex reasoning or planning the approach → think tool
- For issues requiring human judgment or outside system capabilities → transfer_to_human_agents
</delegation_rules>

<instructions>
- DO NOT ASK USER TO CONFIRM WITH MODIFICATION. ASSUME IT IS YES.
- Make sure you handoff to the proper agents
- Remember to check if the the airport city is in the state mentioned by the user. For example, Houston is in Texas.
- Infer about the the U.S. state in which the airport city resides. For example, Houston is in Texas.
- You should not use made-up or placeholder arguments.
- Do not ask for any confirmation from the user. Just go ahead and execute your actions.
- Do not ask the user if they want you to proceed or not. Do not ask user for another card. Just provide information on what they can do next.
</instructions>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt


def supervisor_agent_model():

    model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" #"anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def supervisor_agent_tracing(user_id, session_id, domain):
    trace_attributes = {
        "user.id": user_id, 
        "session.id": session_id,
        "langfuse.tags": [
            user_id,
            session_id,
            f"awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-{domain}",
        ]
    }

    return trace_attributes


def supervisor_swarm_react_agent(tools, user_id, session_id, domain):
    name = f"awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-{domain}-{user_id}-{session_id}",
    prompt = supervisor_agent_prompt()
    model = supervisor_agent_model()
    trace_attributes = supervisor_agent_tracing(user_id, session_id, domain)

    return Agent(
        name = f"awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-{domain}-{user_id}-{session_id}-2",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )

def analyze_airline_with_collaborative_swarm(query: str, user_id, session_id, domain):
    """Collaborative swarm using Nova LITE to avoid streaming timeouts"""
    try:
        reservation_agent = reservation_react_agent(reservation_agent_tools)
        flight_agent = flight_react_agent(flight_agent_tools)
        user_agent = user_react_agent(user_agent_tools)
        supervisor_agent = supervisor_swarm_react_agent(supervisor_tools, user_id, session_id, domain)
        swarm = Swarm(
            [supervisor_agent, user_agent, flight_agent, reservation_agent],
            max_handoffs=15,
            max_iterations=10,
            execution_timeout=300.0,
            node_timeout=300.0
        )
        result = swarm(query)
        # Get performance metrics
        # print(result)
        print(f"Total iterations: {result.execution_count}")
        print(f"Execution time: {result.execution_time}ms")
        print(f"Token usage: {result.accumulated_usage}")
        
        return {
            "status": result.status,
            "result": result
        }
    except Exception as e:
        return {"status": "error", "collaborative_analysis": f"Analysis failed: {str(e)}"}

# Run

In [10]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)

In [12]:
# for index,task in enumerate(tasks):

index = 20 #[10, 14, 19, 20]
task = tasks[index]

index_str = str(index)
num_hashes = (50 - len(index_str) - 9) // 2
print(f"\n{'#' * num_hashes} Index:{index} {'#' * num_hashes}\n")

question = task['question']
print(f"Processing question: {question}")

user_id = task['user_id']
session_id = uuid.uuid4()
print(f"User ID: {user_id}\tSession ID: {session_id}\tDomain:{domain}")

# user = simulated_user(instruction)
# agent = supervisor_react_agent(supervisor_tools, user_id, session_id, domain)
# agent = supervisor_swarm_react_agent(supervisor_tools, user_id, session_id, domain)
messages = analyze_airline_with_collaborative_swarm(question, user_id, session_id, domain)

tool_use_info = {}
# print(messages)
for k in messages["result"].results.keys():
    # print(messages["result"].results[k])
    tool_use_info[k] = []
    for tool_name, tool_use in messages["result"].results[k].result.metrics.tool_metrics.items():
        tool_use_info[k].append({"tool_name": tool_name, "tool_input": tool_use.tool["input"]})

with open(task['user_id']+f"{session_id}.json", "w") as j_file:
    json.dump(tool_use_info, j_file, indent=2)



################### Index:20 ###################

Processing question: My user id is james_taylor_7043. I want to change my upcoming one-stop flight from LAS to IAH to a nonstop flight. My reservation ID is 1N99U6. I also want to remove my checked bag and want the agent to refund me for the same.
User ID: james_taylor_7043	Session ID: 5d246338-8fca-4e7f-aac0-8541b5897430	Domain:airline


DEBUG | strands.multiagent.swarm | nodes=<['awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-airline-james_taylor_7043-5d246338-8fca-4e7f-aac0-8541b5897430-2', 'awsStrands-multiAgent_swarm_singleTurn_userAgent-airline', 'awsStrands-multiAgent_swarm_singleTurn_flightAgent-airline', 'awsStrands-multiAgent_swarm_singleTurn_reservationAgent-airline']> | initialized swarm with nodes
DEBUG | strands.multiagent.swarm | entry_point=<awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-airline-james_taylor_7043-5d246338-8fca-4e7f-aac0-8541b5897430-2> | using first node as entry point
DEBUG | strands.multiagent.swarm | tool_count=<1>, node_count=<4> | injected coordination tools into agents
DEBUG | strands.multiagent.swarm | starting swarm execution
DEBUG | strands.multiagent.swarm | current_node=<awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-airline-james_taylor_7043-5d246338-8fca-4e7f-aac0-8541b5897430-2> | starting swarm execution with node
DEBUG | strands.multiagent.swarm | 

I'll help you change your flight from a one-stop to a nonstop flight and handle the baggage modification. Let me hand this off to the reservation agent who can access your booking details and make the necessary changes.
Tool #1: handoff_to_agent
{
    "name": "chat",
    "context": {
        "trace_id": "0xa1444bb99bc71d8237250b6d2e101eed",
        "span_id": "0x7891ff6cca3599b3",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": "0x8abf35ae6507bd1d",
    "start_time": "2025-10-27T21:54:19.520033Z",
    "end_time": "2025-10-27T21:54:27.359173Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "gen_ai.event.start_time": "2025-10-27T21:54:19.520037+00:00",
        "gen_ai.system": "strands-agents",
        "gen_ai.operation.name": "chat",
        "gen_ai.request.model": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "gen_ai.event.end_time": "2025-10-27T21:54:27.359119+00:00",
        "gen_ai.usage.prompt_tokens": 2968

DEBUG | strands.multiagent.swarm | from_node=<awsStrands-multiAgent_swarm_singleTurn_supervisorAgent-airline-james_taylor_7043-5d246338-8fca-4e7f-aac0-8541b5897430-2>, to_node=<awsStrands-multiAgent_swarm_singleTurn_reservationAgent-airline> | handed off from agent to agent


{
    "name": "execute_tool handoff_to_agent",
    "context": {
        "trace_id": "0xa1444bb99bc71d8237250b6d2e101eed",
        "span_id": "0x1e9829c861c9f2e1",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x8abf35ae6507bd1d",
    "start_time": "2025-10-27T21:54:27.515476Z",
    "end_time": "2025-10-27T21:54:27.517638Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "gen_ai.event.start_time": "2025-10-27T21:54:27.515488+00:00",
        "gen_ai.operation.name": "execute_tool",
        "gen_ai.system": "strands-agents",
        "gen_ai.tool.name": "handoff_to_agent",
        "gen_ai.tool.call.id": "tooluse_71p6kmnSTY2FRmI6T20Ufg",
        "gen_ai.event.end_time": "2025-10-27T21:54:27.517601+00:00",
        "tool.status": "success"
    },
    "events": [
        {
            "name": "gen_ai.tool.message",
            "timestamp": "2025-10-27T21:54:27.515671Z",
            "attributes": {
                "role": "

...

{
    "name": "execute_tool handoff_to_agent",
    "context": {
        "trace_id": "0xa1444bb99bc71d8237250b6d2e101eed",
        "span_id": "0x3093fe664c4fdad1",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x8f75ee8b25251526",
    "start_time": "2025-10-27T21:55:01.643426Z",
    "end_time": "2025-10-27T21:55:01.645727Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "gen_ai.event.start_time": "2025-10-27T21:55:01.643435+00:00",
        "gen_ai.operation.name": "execute_tool",
        "gen_ai.system": "strands-agents",
        "gen_ai.tool.name": "handoff_to_agent",
        "gen_ai.tool.call.id": "tooluse_9ysc9erBQ7aEVlOSKWMSAg",
        "gen_ai.event.end_time": "2025-10-27T21:55:01.645692+00:00",
        "tool.status": "success"
    },
    "events": [
        {
            "name": "gen_ai.tool.message",
            "timestamp": "2025-10-27T21:55:01.643657Z",
            "attributes": {
                "role": "

DEBUG | strands.multiagent.swarm | node=<awsStrands-multiAgent_swarm_singleTurn_flightAgent-airline> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<awsStrands-multiAgent_swarm_singleTurn_reservationAgent-airline>, iteration=<4> | executing node


Okay, let me update the reservation 1N99U6 for james_taylor_7043 to change the flight to a nonstop option from LAS to IAH on 2024-05-19 and remove the checked bag with refund.
Tool #3: get_reservation_details
{
    "name": "chat",
    "context": {
        "trace_id": "0xa1444bb99bc71d8237250b6d2e101eed",
        "span_id": "0x1849e9c3227e1e92",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": "0x508fe9c71c252cb4",
    "start_time": "2025-10-27T21:55:04.910528Z",
    "end_time": "2025-10-27T21:55:08.815554Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "gen_ai.event.start_time": "2025-10-27T21:55:04.910533+00:00",
        "gen_ai.system": "strands-agents",
        "gen_ai.operation.name": "chat",
        "gen_ai.request.model": "anthropic.claude-3-sonnet-20240229-v1:0",
        "gen_ai.event.end_time": "2025-10-27T21:55:08.815477+00:00",
        "gen_ai.usage.prompt_tokens": 5311,
        "gen_ai.usage.input_tokens": 5

DEBUG | strands.multiagent.swarm | node=<awsStrands-multiAgent_swarm_singleTurn_reservationAgent-airline> | node execution completed
DEBUG | strands.multiagent.swarm | node=<awsStrands-multiAgent_swarm_singleTurn_reservationAgent-airline> | no handoff occurred, marking swarm as complete
DEBUG | strands.multiagent.swarm | status=<Status.COMPLETED> | swarm execution completed
DEBUG | strands.multiagent.swarm | node_history_length=<4>, time=<104.90>s | metrics


{
    "name": "invoke_swarm",
    "context": {
        "trace_id": "0xa1444bb99bc71d8237250b6d2e101eed",
        "span_id": "0x4bbf7ecdb9428f14",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": null,
    "start_time": "2025-10-27T21:54:19.514164Z",
    "end_time": "2025-10-27T21:56:04.414252Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "gen_ai.event.start_time": "2025-10-27T21:54:19.514174+00:00",
        "gen_ai.system": "strands-agents",
        "gen_ai.agent.name": "swarm",
        "gen_ai.operation.name": "invoke_swarm"
    },
    "events": [
        {
            "name": "gen_ai.user.message",
            "timestamp": "2025-10-27T21:54:19.514212Z",
            "attributes": {
                "content": "My user id is james_taylor_7043. I want to change my upcoming one-stop flight from LAS to IAH to a nonstop flight. My reservation ID is 1N99U6. I also want to remove my checked bag and want the agent to refu